# Foundation Models for Earth Observation

This notebook demonstrates using embeddings from pre-trained foundation models for crop type classification in agricultural monitoring.

## Learning Objectives
- Load pre-computed embeddings from the Alpha Earth Foundation model
- Visualize high-dimensional embeddings using false-color RGB composites
- Apply unsupervised clustering to discover crop patterns
- Train supervised classifiers for crop type classification with minimal samples
- Test model generalization across geographically distant regions
- Understand few-shot learning and domain adaptation

## Why Crop Classification Matters

Accurate, timely crop mapping is critical for:
- **Food Security**: Monitor crop production and predict yields globally
- **Agricultural Policy**: Support subsidy programs and crop insurance
- **Climate Adaptation**: Track changing agricultural practices
- **Resource Management**: Optimize water use and fertilizer application
- **Trade & Markets**: Forecast commodity supplies

Traditional crop mapping requires:
- Extensive field surveys (expensive, slow)
- Expert image interpretation (not scalable)
- Lots of labeled training data (hard to collect)

Foundation models enable rapid, scalable crop mapping with minimal ground truth!

## What are Foundation Model Embeddings?

Foundation models are large neural networks pre-trained on massive amounts of satellite imagery (millions of images globally). They learn to extract meaningful features without task-specific labels.

**Key Concepts:**
- **Embeddings**: Compact numerical representations (64 dimensions) that capture patterns in imagery
- **Transfer Learning**: Knowledge learned from global data transfers to your specific region
- **Few-Shot Learning**: Achieve good results with just a handful of labeled samples per class

**Alpha Earth Foundation Model:**
We'll use [Google Alpha Earth Embeddings](https://arxiv.org/pdf/2507.22291). Note: Not all foundation models are created equal! Always understand:
- What data was used for pre-training?
- What geographic regions and time periods?
- What biases might exist?
- What are the model's limitations?

## Setup

Install required packages and import helper functions.

In [ ]:
# Install dependencies
%pip install numpy pandas matplotlib rasterio seaborn xarray pyproj dask rioxarray pystac-client planetary-computer scikit-learn pyarrow tqdm scipy leafmap -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    !git clone https://github.com/rohansaw/FoundationModels4EO.git
    %cd FoundationModels4EO
else:
    print("Running locally - skipping git clone")

# Import our helper functions
from geo_helpers import (
    load_sentinel2_rgb,
    load_sentinel2_rgb_timeseries,
    load_crop_labels,
    load_foundation_model_embeddings,
    create_embedding_rgb,
    simplify_crop_labels,
    align_labels_to_embeddings,
    prepare_training_data,
    get_class_names,
    print_crop_statistics,
    prepare_csv_samples,
    show_split_statistics,
    compute_clusters,
    predict_on_embeddings,
    prepare_sample_coordinates
)

from viz_helpers import (
    plot_rgb_image,
    plot_rgb_timeseries,
    plot_crop_labels,
    plot_embeddings_rgb,
    plot_clustering_results,
    plot_classification_results,
    plot_prediction_map,
    plot_generalization_comparison,
    show_study_area_map,
    plot_classification_vs_clustering,
    plot_sample_map_by_class,
    plot_sample_map_by_split,
    plot_classification_vs_clustering
)

## 1. Define Study Area

**Background:** The Mississippi Delta is one of the most productive agricultural regions in the United States, known for:
- Extensive corn and soybean production
- Flat terrain ideal for mechanized farming
- Rich soils from the Mississippi River
- Humid subtropical climate with long growing season

**Your Task:**
- Run the cell below to visualize our study area
- Examine the interactive map
- Note the agricultural landscape patterns

In [ ]:
# Mississippi Delta region - corn/soy agricultural area
LON_MIN, LON_MAX = -90.90, -90.75
LAT_MIN, LAT_MAX = 33.45, 33.60
YEAR = 2024

# Visualize the study area on an interactive map
print("Study Area")
show_study_area_map(LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)

## 2. Load Satellite Imagery and Embeddings

We'll load two types of data:

**1. Sentinel-2 L2A RGB Imagery:**
- Free, open satellite imagery from the European Space Agency
- 10-meter spatial resolution
- Revisit every 5 days (with two satellites)
- Loaded from [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) via STAC API

**2. Google Alpha Earth Foundation Model Embeddings:**
- Pre-computed 64-dimensional feature vectors for each pixel
- Loaded from [Source Cooperative](https://source.coop/tge-labs/aef) (Courtesy of [Taylor Geospatial Engine Labs](https://tgengine.org/))

**Your Task:**
- Run the cells to load both datasets
- Compare RGB imagery across the growing season
- Note how embeddings compress complex patterns into 64 numbers

In [ ]:
# Load Sentinel-2 RGB imagery for growing season
# Mississippi growing season: April-October for corn/soy
GROWING_SEASON_MONTHS = [4, 5, 6, 7, 8, 9, 10]  # Apr-Oct

rgb_timeseries = load_sentinel2_rgb_timeseries(
    LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, YEAR, GROWING_SEASON_MONTHS
)

plot_rgb_timeseries(rgb_timeseries, "Mississippi Delta", YEAR)

In [ ]:
# Load foundation model embeddings (64-dimensional feature vector for each pixel)
# This may take up to 3 minutes depending on your internet connection
embeddings = load_foundation_model_embeddings(LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, YEAR)

print(f"Embedding shape: {embeddings.shape}")
print(f"  - 64 feature dimensions capturing crop characteristics")
print(f"  - {embeddings.shape[1]} x {embeddings.shape[2]} pixels")


## 3. Visualize Embeddings

**Challenge:** Embeddings have 64 dimensions, but we can only see in 3D (RGB)!

**Solution:** Create a false-color RGB composite by selecting 3 embedding dimensions and mapping them to red, green, and blue channels.

**What to look for:**
- Different crops may appear in different colors
- Field boundaries and patterns
- How different dimensions highlight different features

**Your Task:**
- Run the cell with default dimensions [0, 10, 2]
- **Experiment:** Try different dimension combinations
  - Example: [5, 15, 25], [1, 30, 50], [10, 20, 40]. What do you note?

In [ ]:
# Create RGB visualization from 3 embedding dimensions
dimensions_to_visualize = [0, 10, 2]  # Try changing these!

embedding_rgb = create_embedding_rgb(embeddings, bands=dimensions_to_visualize)
plot_embeddings_rgb(embedding_rgb, bands=dimensions_to_visualize)

**Reflection Questions:**
1. How does the embedding visualization compare to the RGB satellite imagery?
2. Can you identify different crop types based on color patterns?
3. Are field boundaries clearly visible?
4. What do you think each dimension might be capturing?

## 4. Unsupervised Clustering

**What is Clustering?**
Clustering groups similar pixels together without using labels. It's completely unsupervised - the algorithm discovers natural patterns in the data.

**Why use clustering?**
- Explore data before labeling
- Discover unexpected patterns
- Validate that different crops have distinct signatures
- Create initial labels for semi-supervised learning

**What patterns might emerge?**
- Fields with the same crop type
- Built-up areas vs vegetation
- Water bodies
- Different crop growth stages
- Soil types

**Your Task:**
- Run clustering with k=[3, 5, 10]
- **Experiment:** Try different numbers of clusters (e.g., [3, 7, 15])
- More clusters = finer distinctions but longer processing time
- Are you able to identify what different clusters represent?

In [ ]:
# Try k-means clustering with different numbers of clusters
number_of_clusters_to_explore = [3, 5, 10]  # Try changing this!

# Compute clusters
cluster_results = compute_clusters(embeddings, k_values=number_of_clusters_to_explore)

# Plot results
plot_clustering_results(cluster_results=cluster_results)

**Analysis Questions:**
1. What happens when you increase the number of clusters?
2. With k=3, can you identify broad categories (vegetation vs non-vegetation)?
3. With k=10, do you see finer distinctions between crop types?
4. Do cluster boundaries align with field boundaries?
5. What are the limitations of unsupervised clustering for crop classification?
6. Would you need labeled data to use these clusters for crop mapping?

## 5. Supervised Learning: Train a Crop Classifier

**From Unsupervised to Supervised:**
Clustering revealed patterns, but we need labels to identify specific crops. Now we'll train a supervised classifier using labeled samples.

**About the Training Data:**
- Pre-extracted AlphaEarth embeddings for point locations across Mississippi
- Labels from USDA Cropland Data Layer (CDL)
- **Important:** CDL is model-predicted, not field-validated ground truth
- 1,000 samples per class available
- We'll use only 20 training samples per class to demonstrate few-shot learning!

**Three Classes:**
- **Corn**: Major grain crop
- **Soy (Soybeans)**: Major oilseed and protein crop
- **Other**: Everything else (other crops, forest, urban, water, etc.)

**Your Task:**
- Load the pre-labeled CSV data
- Examine the class distribution
- Prepare balanced train and test splits

In [ ]:
# Load pre-extracted Mississippi embeddings

samples_mississipi = pd.read_csv("demo_data/mississippi_alphaearth_2024.csv")
print(f"Loaded {len(samples_mississipi)} samples with embeddings")
print(f"\nClasses: {samples_mississipi['class_name'].value_counts().to_dict()}")

In [ ]:
# Prepare training and test splits
# TODO: Experiment with different numbers of training samples!
NUM_SAMPLES = {
    'corn': {'label': 0, 'n_train': 20, 'n_test': 100},
    'soy': {'label': 1, 'n_train': 20, 'n_test': 100},
    'other': {'label': 2, 'n_train': 20, 'n_test': 50}
}

X_train, y_train, X_test, y_test, train_idx, test_idx = \
    prepare_csv_samples(samples_mississipi, NUM_SAMPLES, random_seed=42)

show_split_statistics(samples_mississipi, train_idx, test_idx)

### Spatial Distribution of Samples

Understanding spatial distribution matters:
- Identifying spatial autocorrelation (nearby samples are often similar)
- Detecting spatial bias
- Understanding coverage of training vs test data

We'll create interactive maps showing samples and their train/test split.


In [ ]:
# Prepare sample coordinates for visualization
samples_mississipi = prepare_sample_coordinates(samples_mississipi, train_idx, test_idx)

In [ ]:
# Visualize all samples by crop class
m_all = plot_sample_map_by_class(samples_mississipi, center_lat=33.5, center_lon=-90.8, zoom=7)
m_all

In [ ]:
# Visualize train vs test split
m_split = plot_sample_map_by_split(samples_mississipi, center_lat=33.5, center_lon=-90.8, zoom=7)
m_split

### Understanding Spatial Autocorrelation

**Tobler's First Law of Geography:**
> "Everything is related to everything else, but near things are more related than distant things"

**Spatial autocorrelation** means observations at nearby locations tend to be similar.

**Why this matters for machine learning:**
1. **Inflated accuracy metrics**: Train/test samples close together make models look better than they are
2. **Location-specific learning**: Models may learn local patterns that don't generalize
3. **Violates i.i.d. assumption**: Standard ML assumes independent, identically distributed samples
4. **Overestimates generalization**: Model may fail in new regions despite high validation accuracy

**In agricultural classification:**
- Nearby fields often have the same crop (farm management patterns)
- Environmental conditions (soil, climate) vary smoothly across space
- Regional agricultural practices and crop rotations
- Satellite imagery has spatial correlation in adjacent pixels

Keep this in mind when working with ML solutions in the Geo Field!

In [ ]:
# Train classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("Training Random Forest classifier...")
print(f"Using only {len(X_train)} training samples (20 per class)!")
print("This demonstrates the power of foundation model embeddings.\n")

clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2%}")

# Show results
plot_classification_results(y_test, y_pred, accuracy, ['other', 'corn', 'soy'])

**Analysis Questions:**
1. How well does the model perform with only 20 training samples per class?
2. Which classes are most confused with each other? (Look at the confusion matrix)
3. Why might "other" be harder to classify than corn or soy?
4. How many samples would traditional ML (without foundation models) need?
5. What does this tell you about the quality of the embeddings?

## 6. Apply Classifier to Our Study Area

**From Points to Maps:**
So far we've classified individual sample points. Now let's create a wall-to-wall crop map for the entire study area!

**What we'll do:**
1. Apply our trained classifier to every pixel's embedding
2. Generate a classified crop map
3. Compare with unsupervised clustering results

**Your Task:**
- Run the prediction
- Examine the output statistics (% of each crop)
- Compare the supervised map with clustering from earlier
- Look for patterns: field boundaries, roads, crop distribution

In [ ]:
# Apply classifier to spatial embeddings from our study area

print("Predicting crop types for study area...")
crop_predictions = predict_on_embeddings(clf, embeddings)

print(f" Predictions complete!")
print(f"Map shape: {crop_predictions.shape}")
print(f"\nPredicted crop distribution:")
for label, name in enumerate(['other', 'corn', 'soy']):
    count = np.sum(crop_predictions == label)
    pct = 100 * count / crop_predictions.size
    print(f"  {name.capitalize()}: {pct:.1f}%")

In [ ]:
# Compare classification with clustering using precomputed clusters

print("Comparing supervised classification with unsupervised clustering...")

# Display side-by-side comparison
plot_classification_vs_clustering(
    embeddings, 
    None,
    crop_predictions, 
    {5: cluster_results[5]},  # Only use k=5 cluster from precomputed results
    class_names=['other', 'corn', 'soy']
)

**Understanding Limitations**

**Observation:** The unsupervised clustering looks cleaner - field borders and roads are clearly visible. Why doesn't the supervised model show the same clarity?

**Root Cause - Training Data Quality:**
This highlights a critical lesson: **understand your training data, not just validation metrics!**

Our labeled samples come from the **USDA Cropland Data Layer (CDL)**:
- 30m spatial resolution (coarser than our 10m embeddings)
- Roads and narrow features aren't always clearly visible
- Road pixels often merged with adjacent field pixels
- The model never learned to separate roads from crops

**Result:** The classifier likely labels road pixels as "corn" / "soy" / rest based on nearby fields.

**Key Takeaway:** 
- High accuracy on test data doesn't mean perfect predictions everywhere
- Training data quality and resolution matter enormously
- Always validate predictions against ground truth or high-resolution imagery
- Understand the provenance and limitations of your labels

**In practice, you should:**
- Use higher-resolution ground truth when possible
- Validate predictions with field data
- Be aware of label noise and its propagation
- Consider active learning to improve labels iteratively

## 7. Testing Model Generalization Across Regions

### Can Foundation Models Generalize?

We trained our classifier with samples from Mississippi and achieved great results! But in real-world applications (especially disaster response), we often need models to work in completely new regions.

**The Challenge:** Mississippi and Minnesota are ~1,400km apart with very different environmental conditions:

| Factor | Mississippi | Minnesota |
|--------|-------------|-----------|
| **Latitude** | 30.2N - 35.0°N | 43.5°N - 49.4°N |
| **Climate** | Humid Subtropical | Continental |
| **Growing Season** | 240-270 days | 120-160 days |
| **Summer Temperature** | 27-29°C | 20-23°C |
| **Winter Temperature** | 8-11°C | -12 to -7°C |
| **Annual Rainfall** | 1,400-1,600 mm | 500-750 mm |
| **Soil Type** | Clay loam, alluvial | Mollisols, glacial till |

**Why this matters for ML:**
- Different spectral signatures due to climate
- Different crop phenology (growth timing)
- Different stress patterns (heat vs cold)
- Testing true generalization capability

These differences create a "domain shift" - can our model handle it? Let's find out!

### Load Minnesota Data for Testing

In [ ]:
# Load Minnesota data
samples_minnesota = pd.read_csv("demo_data/minnesota_alphaearth_2024.csv")

# Prepare test set (300 samples per class)
embedding_columns = [col for col in samples_minnesota.columns if col.startswith('A')]
X_minnesota_all = samples_minnesota[embedding_columns].values
y_minnesota_all = samples_minnesota['label'].values

# Sample test set
np.random.seed(42)
test_indices_mn = []
for label in [0, 1, 2]:
    class_idx = np.where(y_minnesota_all == label)[0]
    sampled = np.random.choice(class_idx, min(300, len(class_idx)), replace=False)
    test_indices_mn.extend(sampled)

test_indices_mn = np.array(test_indices_mn)
np.random.shuffle(test_indices_mn)

X_test_minnesota = X_minnesota_all[test_indices_mn]
y_test_minnesota = y_minnesota_all[test_indices_mn]

print(f"Minnesota test set: {len(X_test_minnesota)} samples")
for label in [0, 1, 2]:
    count = np.sum(y_test_minnesota == label)
    name = samples_minnesota[samples_minnesota['label'] == label]['class_name'].iloc[0]
    print(f"  {name}: {count} samples")

In [ ]:
# Test Mississippi-trained model on Minnesota (Zero-Shot Transfer)
print("Testing zero-shot transfer: Mississippi model -> Minnesota data")
print("="*60)

y_pred_zeroshot = clf.predict(X_test_minnesota)
accuracy_zeroshot = accuracy_score(y_test_minnesota, y_pred_zeroshot)

print(f"\n Zero-Shot Accuracy: {accuracy_zeroshot:.2%}")
print(f"\n  Trained on: {len(X_train)} Mississippi samples")
print(f"  Tested on: {len(X_test_minnesota)} Minnesota samples")
print(f"  Distance: ~1,400 km apart")

# Show results
plot_classification_results(y_test_minnesota, y_pred_zeroshot, accuracy_zeroshot, 
                           ['other', 'corn', 'soy'])

### Few-Shot Adaptation

**The Challenge:**
Performance dropped significantly due to **distribution shift** (domain shift) between Mississippi and Minnesota:
- Different climate and growing conditions
- Different crop phenology (growth timing and patterns)
- Different spectral signatures
- Different agricultural practices

**The Question:**
Can we improve performance by adding just a few samples from the target region?

**Few-Shot Learning:**
Let's add only **5 Minnesota samples per class** (15 total) to our training set and see what happens!

This simulates a real-world scenario:
- Limited time/budget for field surveys in new region
- Need rapid deployment (disaster response, early warning)
- Foundation models should make this efficient

**Your Task:**
- Run the few-shot adaptation
- Compare zero-shot vs few-shot accuracy
- Analyze the improvement
- Consider: Is 5 samples per class reasonable to collect in practice?

In [ ]:
# Add 5 Minnesota samples per class to training set
N_FEWSHOT = 5

# Get samples not in test set
remaining_idx = np.setdiff1d(np.arange(len(y_minnesota_all)), test_indices_mn)

fewshot_indices = []
for label in [0, 1, 2]:
    class_remaining = remaining_idx[y_minnesota_all[remaining_idx] == label]
    sampled = np.random.choice(class_remaining, N_FEWSHOT, replace=False)
    fewshot_indices.extend(sampled)

fewshot_indices = np.array(fewshot_indices)

X_fewshot = X_minnesota_all[fewshot_indices]
y_fewshot = y_minnesota_all[fewshot_indices]

# Combine with Mississippi training data
X_train_combined = np.vstack([X_train, X_fewshot])
y_train_combined = np.concatenate([y_train, y_fewshot])

print(f"Combined training set:")
print(f"  Mississippi: {len(X_train)} samples")
print(f"  Minnesota: {len(X_fewshot)} samples")
print(f"  Total: {len(X_train_combined)} samples")

In [ ]:
# Train combined model
print("Training model with Mississippi + Minnesota few-shot samples...")

clf_combined = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf_combined.fit(X_train_combined, y_train_combined)

# Test on Minnesota
y_pred_fewshot = clf_combined.predict(X_test_minnesota)
accuracy_fewshot = accuracy_score(y_test_minnesota, y_pred_fewshot)

print(f"\n Few-Shot Accuracy: {accuracy_fewshot:.2%}")
print(f"  Improvement: {accuracy_fewshot - accuracy_zeroshot:+.2%}")

# Show results
plot_classification_results(y_test_minnesota, y_pred_fewshot, accuracy_fewshot,
                           ['other', 'corn', 'soy'])

In [ ]:
# Visualize comparison
from sklearn.metrics import confusion_matrix
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

class_names = ['other', 'corn', 'soy']

# Zero-shot confusion matrix
cm_zero = confusion_matrix(y_test_minnesota, y_pred_zeroshot)
sns.heatmap(cm_zero, annot=True, fmt='d', cmap='Reds', ax=axes[0],
            xticklabels=class_names, yticklabels=class_names)
axes[0].set_title(f'Zero-Shot Transfer\n(Mississippi only)\nAccuracy: {accuracy_zeroshot:.2%}',
                  fontsize=13, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Few-shot confusion matrix
cm_few = confusion_matrix(y_test_minnesota, y_pred_fewshot)
sns.heatmap(cm_few, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=class_names, yticklabels=class_names)
axes[1].set_title(f'Few-Shot Adapted\n(+5 MN samples/class)\nAccuracy: {accuracy_fewshot:.2%}',
                  fontsize=13, fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

**Analysis Questions:**

1. **Zero-Shot Performance:**
   - How well did the Mississippi-trained model work on Minnesota?
   - Which classes had the most errors?
   - Why didn't it completely fail despite the domain shift?

2. **Few-Shot Improvement:**
   - How much did accuracy improve with just 5 samples per class?
   - Is this improvement practically significant?
   - Which classes benefited most from the few-shot samples?

3. **Practical Implications:**
   - Could you collect 15 samples (5 per class) in a day of field work?
   - How does this compare to traditional approaches needing hundreds of samples?
   - When would few-shot learning be especially valuable?

4. **Foundation Model Benefits:**
   - Why can foundation models learn from so few samples?
   - What knowledge transferred from Mississippi to Minnesota?
   - What had to be adapted with the few-shot samples?

## Key Takeaways

### What We Learned About Foundation Models

1. **Rich Embeddings Capture Crop Identity**
   - 64-dimensional embeddings encode meaningful crop characteristics
   - Pre-training on massive global datasets creates generalizable features
   - Unsupervised clustering reveals natural patterns aligned with crop types
   - Can be visualized as false-color images to interpret what the model "sees"

2. **Efficient Training with Minimal Samples**
   - Achieved high accuracy with only **20 training samples per class**
   - Foundation models transfer knowledge from pre-training on millions of images
   - Dramatically reduces field survey costs and time

3. **Zero-Shot Transfer Works Across Regions**
   - Model trained on Mississippi performed reasonably on Minnesota
   - **~1,400km distance** with very different environmental conditions
   - Demonstrates that foundation models learn generalizable crop features
   - Not perfect, but provides a strong starting point for new regions

4. **Few-Shot Adaptation is Powerful**
   - Adding just **5 samples per class (15 total)** significantly improved performance
   - Orders of magnitude less data than traditional approaches
   - Critical for rapid deployment in new regions
   - Enables agile response to emerging food security challenges

### Practical Applications

**Disaster Response & Early Warning:**
- Quickly map crops in disaster-affected regions
- Minimal field surveys needed
- Rapid deployment for damage assessment

**Global Agricultural Monitoring:**
- Train models in data-rich areas
- Deploy to data-poor regions worldwide
- Monitor food production globally

**Cost Efficiency:**
- Reduce field campaign costs by 90%+
- Smaller teams can cover larger areas
- More frequent monitoring becomes feasible

**Rapid Prototyping:**
- Test classification approaches quickly
- Validate feasibility before full data collection
- Iterate on class definitions efficiently

**Validation Strategy:**
- This demo used simplified train/test splits
- Real applications should use train/validation/test or k-fold cross-validation
- Validate hyperparameters on separate data

**Ground Truth Quality:**
- CDL is predicted data, not perfect ground truth
- Label quality directly impacts model performance
- Always validate with field data when possible
- Consider label noise in your accuracy expectations

**Foundation Model Limitations:**
- Pre-training data may not cover all regions equally
- Temporal coverage varies
- Biases exist based on training data and generalization can be limited!
- Always validate in your specific context

### Reflections & Discussion

**Think about:**

1. **Data Efficiency:**
   - How would your workflow change with foundation models vs traditional ML?
   - What field surveys become feasible that weren't before?
   - How does this impact global food security monitoring?

2. **Generalization:**
   - When would zero-shot transfer work well vs poorly?
   - What factors determine if few-shot learning is sufficient?
   - How would you decide how many samples to collect?

3. **Model Selection:**
   - When would you use clustering vs supervised learning?
   - Could you combine both approaches?
   - What other classifiers might work better than Random Forest?

4. **Operational Deployment:**
   - How would you validate predictions before making decisions?
   - What quality control steps would you implement?
   - How would you handle prediction uncertainty?


### Resources for Further Learning

**Foundation Models:**
- [Alpha Earth Paper](https://arxiv.org/pdf/2507.22291): Technical details and benchmarks
- [Prithvi](https://huggingface.co/ibm-nasa-geospatial): NASA-IBM geospatial foundation model
- [Segment Anything Model](https://segment-anything.com/): For image segmentation tasks

**Data Sources:**
- [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/): Free satellite data access
- [Google Earth Engine](https://earthengine.google.com/): Petabyte-scale analysis platform
- [Source Cooperative](https://source.coop/): Open geospatial data repository

**Agricultural Monitoring:**
- [USDA NASS](https://www.nass.usda.gov/): Official US crop statistics
- [GEOGLAM](https://www.geoglam.org/): Global agricultural monitoring initiative
- [Crop Monitor](https://cropmonitor.org/): Monthly crop conditions worldwide

**Machine Learning:**
- [Scikit-learn](https://scikit-learn.org/): ML library documentation
